In [25]:
import polars as pl
import requests
import shapely

In [26]:
r = requests.get(
    "http://bustime.mta.info/api/where/vehicles-for-agency/MTABC.json?key=OBANYC"
).json()
df_oba = pl.from_dicts(r["data"]["list"])
r = requests.get(
    "http://bustime.mta.info/api/where/vehicles-for-agency/MTA NYCT.json?key=OBANYC"
).json()
# combine the two dataframes
df_oba = pl.concat([df_oba, pl.from_dicts(r["data"]["list"])])

In [29]:
r = requests.get(
    "https://bustime.mta.info/api/siri/vehicle-monitoring.json?key=OBANYC&version=2"
).json()
parsed = [
    v["MonitoredVehicleJourney"]
    for v in r["Siri"]["ServiceDelivery"]["VehicleMonitoringDelivery"][0][
        "VehicleActivity"
    ]
]
df_siri = pl.from_dicts(
    parsed
    # r["Siri"]["ServiceDelivery"]["VehicleMonitoringDelivery"][0]["VehicleActivity"]
)

In [30]:
df_siri = df_siri.with_columns(
    pl.col("VehicleRef").str.split("_").list.last().alias("vehicle_id")
)
df_oba = df_oba.with_columns(
    pl.col("vehicleId").str.split("_").list.last().alias("vehicle_id")
)
# [
#     "VehicleID"
# ]
# create polars series with vehicle IDs

In [31]:
display(df_oba, df_siri)

lastLocationUpdateTime,lastUpdateTime,location,occupancyCapacity,occupancyCount,occupancyStatus,phase,status,tripId,vehicleId,vehicle_id
i64,i64,struct[2],i64,i64,i64,str,str,str,str,str
1748556018286,1748556018286,"{40.67074,-73.820841}",80,8,1,"""in_progress""","""blockInf""","""MTABC_42507323-JKPB5-JK_B5-Wee…","""MTABC_9128""","""9128"""
1748556027304,1748556027304,"{40.677197,-73.825248}",null,null,null,"""in_progress""","""blockInf""","""MTABC_42505703-JKPB5-JK_B5-Wee…","""MTABC_9129""","""9129"""
1748556017000,1748556017000,"{40.706605,-73.83804}",80,27,1,"""in_progress""","""blockInf""","""MTABC_42507311-JKPB5-JK_B5-Wee…","""MTABC_9126""","""9126"""
1748556023000,1748556023000,"{40.701889,-73.804176}",80,13,1,"""deadhead_during""","""blockInf""","""MTABC_42506271-JKPB5-JK_B5-Wee…","""MTABC_9127""","""9127"""
1748556031362,1748556031362,"{40.678789,-73.833731}",null,null,null,"""in_progress""","""blockInf""","""MTABC_42506390-JKPB5-JK_B5-Wee…","""MTABC_9124""","""9124"""
…,…,…,…,…,…,…,…,…,…,…
1748556044000,1748556044000,"{40.711751,-74.010331}",65,4,1,"""in_progress""","""blockInf""","""MTA NYCT_YU_B5-Weekday-SDon-10…","""MTA NYCT_2545""","""2545"""
1748556024000,1748556024000,"{40.656158,-74.005615}",null,null,null,"""deadhead_during""","""default""","""MTA NYCT_YU_B5-Weekday-SDon-09…","""MTA NYCT_2546""","""2546"""
1748556040000,1748556040000,"{40.646243,-74.016172}",null,null,null,"""in_progress""","""blockInf""","""MTA NYCT_YU_B5-Weekday-SDon-10…","""MTA NYCT_2547""","""2547"""


LineRef,DirectionRef,FramedVehicleJourneyRef,JourneyPatternRef,PublishedLineName,OperatorRef,OriginRef,DestinationRef,DestinationName,Monitored,VehicleLocation,Bearing,ProgressRate,Occupancy,BlockRef,VehicleRef,MonitoredCall,SituationRef,OriginAimedDepartureTime,ProgressStatus,vehicle_id
str,str,struct[2],str,list[str],str,str,str,list[str],bool,struct[2],f64,str,str,str,str,struct[10],list[struct[1]],str,list[str],str
"""MTA NYCT_BX38""","""0""","{""2025-05-29"",""MTA NYCT_GH_B5-Weekday-SDon-107500_BX256_510""}","""MTA_BX380150""","[""Bx38""]","""MTA NYCT""","""MTA_100652""","""MTA_802024""","[""CO-OP CITY BAY PLAZA via GUNHILL""]",true,"{-73.875597,40.879685}",334.57312,"""normalProgress""","""seatsAvailable""","""MTA NYCT_GH_B5-Weekday-SDon_E_…","""MTA NYCT_9771""","{""2025-05-29T18:06:50.002-04:00"",""2025-05-29T18:03:23.030-04:00"",""< 1 stop away"",""2025-05-29T18:03:23.030-04:00"",236,0,{{21,80,""manySeatsAvailable""}},""MTA_101863"",1,[""EAST GUN HILL RD/HULL AV""]}",null,null,null,"""9771"""
"""MTA NYCT_BX28""","""0""","{""2025-05-29"",""MTA NYCT_GH_B5-Weekday-SDon-107100_BX238_630""}","""MTA_BX280725""","[""Bx28""]","""MTA NYCT""","""MTA_103042""","""MTA_101732""","[""CO-OP CITY EARHART LANE via GUNHILL""]",true,"{-73.894035,40.864878}",66.2505,"""normalProgress""","""seatsAvailable""","""MTA NYCT_GH_B5-Weekday-SDon_E_…","""MTA NYCT_9770""","{""2025-05-29T17:52:34.017-04:00"",""2025-05-29T18:01:57.985-04:00"",""< 1 stop away"",""2025-05-29T18:01:57.985-04:00"",169,0,{{3,80,""manySeatsAvailable""}},""MTA_102968"",1,[""EAST KINGSBRIDGE RD/GRAND CONCOURSE""]}",null,null,null,"""9770"""
"""MTA NYCT_Q85""","""1""","{""2025-05-29"",""MTA NYCT_JA_B5-Weekday-SDon-105900_MISC_793""}","""MTA_Q850343""","[""Q85""]","""MTA NYCT""","""MTA_503105""","""MTA_505264""","[""JAMAICA PARSONS-ARCHER""]",true,"{-73.766576,40.675425}",148.86102,"""normalProgress""",null,"""MTA NYCT_JA_B5-Weekday-SDon_E_…","""MTA NYCT_8440""","{""2025-05-29T18:03:36.032-04:00"",""2025-05-29T18:02:04.355-04:00"",""< 1 stop away"",""2025-05-29T18:02:04.355-04:00"",176,0,null,""MTA_500671"",1,[""BEDELL ST/133 AV""]}","[{""MTA NYCT_lmm:planned_work:18177""}, {""MTABC_lmm:planned_work:23593""}]",null,null,"""8440"""
"""MTA NYCT_Q4""","""0""","{""2025-05-29"",""MTA NYCT_JA_B5-Weekday-SDon-108100_Q4_105""}","""MTA_Q040073""","[""Q4""]","""MTA NYCT""","""MTA_504470""","""MTA_904026""","[""CAMBRIA HEIGHTS 235 ST via LINDEN""]",true,"{-73.801094,40.702152}",15.01294,"""normalProgress""",null,"""MTA NYCT_JA_B5-Weekday-SDon_E_…","""MTA NYCT_8443""","{""2025-05-29T18:03:00.043-04:00"",null,""at stop"",null,22,0,null,""MTA_504470"",1,[""ARCHER AV/PARSONS BLVD""]}","[{""MTA NYCT_lmm:planned_work:18177""}, {""MTABC_lmm:planned_work:23593""}]",null,null,"""8443"""
"""MTA NYCT_M2""","""0""","{""2025-05-29"",""MTA NYCT_MV_B5-Weekday-SDon-105500_M3_316""}","""MTA_M020092""","[""M2""]","""MTA NYCT""","""MTA_405527""","""MTA_903049""","[""LIMITED WASHINGTON HTS BROADWAY-168 ST""]",true,"{-73.99031,40.731588}",75.63648,"""noProgress""",null,"""MTA NYCT_MV_B5-Weekday-SDon_E_…","""MTA NYCT_9775""","{""2025-05-29T17:38:30.070-04:00"",""2025-05-29T18:03:32.129-04:00"",""< 1 stop away"",""2025-05-29T18:03:32.129-04:00"",457,0,null,""MTA_404120"",1,[""UNION SQ EAST/E 15 ST""]}","[{""MTA NYCT_lmm:planned_work:23994""}, {""MTA NYCT_lmm:planned_work:23502""}]","""2025-05-29T17:35:00.000-04:00""","[""layover""]","""9775"""
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""MTABC_Q10""","""1""","{""2025-05-29"",""MTABC_42506878-JKPB5-JK_B5-Weekday-02-SDon""}","""MTA_Q100594""","[""Q10""]","""MTABC""","""MTA_550378""","""MTA_550290""","[""150 AV via LEFFERTS BL via 130 ST""]",true,"{-73.826172,40.690356}",296.60953,"""normalProgress""",null,"""MTABC_JKPB5-JK_B5-Weekday-02-S…","""MTABC_5376""","{""2025-05-29T18:04:07.700-04:00"",""2025-05-29T18:01:54.277-04:00"",""approaching"",""2025-05-29T18:01:54.277-04:00"",35,0,null,""MTA_550324"",1,[""LEFFERTS BLVD/101 AV""]}","[{""MTABC_lmm:planned_work:2359

In [33]:
siri_vehicle_ids = df_siri["vehicle_id"]
oba_vehicle_ids = df_oba["vehicle_id"]


display(
    "OBA difference:", oba_vehicle_ids.filter(~oba_vehicle_ids.is_in(siri_vehicle_ids))
)
# display("difference:", siri_vehicle_ids.filter(~siri_vehicle_ids.is_in(oba_vehicle_ids)))

'OBA difference:'

vehicle_id
str
"""9127"""
"""9120"""
"""6090"""
"""6092"""
"""9119"""
…
"""2559"""
"""2551"""
"""2542"""


In [35]:
filtered_df = df_oba.filter(
    pl.col("lastLocationUpdateTime") != pl.col("lastUpdateTime")
)
# rows are equal
# print(f"Total rows: {len(df)}")
# print(f"Filtered rows: {len(filtered_df)}")
# filtered_df.head(5)
filtered_df

lastLocationUpdateTime,lastUpdateTime,location,occupancyCapacity,occupancyCount,occupancyStatus,phase,status,tripId,vehicleId,vehicle_id
i64,i64,struct[2],i64,i64,i64,str,str,str,str,str


In [38]:
df_oba["status"].value_counts()

status,count
str,u32
"""blockInf""",3719
"""default""",1112
"""blockInf+stalled""",10
"""stalled""",11
"""blockInf+deviated""",14


In [37]:
df_siri["ProgressRate"].value_counts()

ProgressRate,count
str,u32
"""normalProgress""",3203
"""noProgress""",318
"""unknown""",1


In [39]:
df_siri["ProgressStatus"].value_counts()

ProgressStatus,count
list[str],u32
"[""layover""]",309
null,3212
"[""spooking""]",1
